In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.image import ImagePromptTemplate
from langchain.prompts import PromptTemplate
from langchain_core.prompt_values import ImageURL
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser

GOOGLE_API_KEY = ""

urls = ['https://img.maisonkorea.com/2020/03/msk_5e65a1179ab47.jpg',
       'https://upload.wikimedia.org/wikipedia/commons/thumb/c/cf/Wohnzimmer_2007.jpg/1200px-Wohnzimmer_2007.jpg',
       'https://static.hyundailivart.co.kr/upload_mall/board/ME00000044/B200042482/tplt/0000215270_20220224105746834.jpg']

In [2]:
class TagsForBoundingBoxes(BaseModel):
    
    detected_furnitures: list[str] = Field(..., description = "List of detected Furnitures");
    
    upper_left_axis_x: list[tuple([float, float])] = Field(..., description = "Upper-left axis of bounding boxes of detected furniture");
    upper_left_axis_y: list[tuple([float, float])] = Field(..., description = "Upper-left axis of bounding boxes of detected furniture");

    bottom_right_axis_x: list[tuple([float, float])] = Field(..., description = "Bottom-right axis of bounding boxes of detected furniture");
    bottom_right_axis_y: list[tuple([float, float])] = Field(..., description = "Bottom-right axis of bounding boxes of detected furniture");

class BoundingBoxGenerator:

    def __init__(self):
        
        self.GOOGLE_API_KEY = GOOGLE_API_KEY
        self.llm = ChatGoogleGenerativeAI(model = 'gemini-pro', google_api_key = self.GOOGLE_API_KEY,
                                         temperature = 0)
        self.lmm = ChatGoogleGenerativeAI(model = 'gemini-pro-vision', google_api_key = self.GOOGLE_API_KEY,
                                         temperature = 0)

        self.parser = PydanticOutputParser(pydantic_object = TagsForBoundingBoxes)
        self.prompt = PromptTemplate(
            template = """Allocated raw query to formatted instruction \n {format_instructions}\n{query}
            Note that the sequence formed like this : something | x, y | z, w
            """,
            input_variables = ["query"],
            partial_variables = {"format_instructions" : self.parser.get_format_instructions()}
        )

    def query_img(self, url:str = None):

        self.message = HumanMessage(
            content = [
                {'type': 'text',
                 'text': """You detect objects in the given image and provide their bounding box coordinates, but you must follow these rules:

                1. Do not suggest all objects; only include those that could be classified to **furniture** or **home appliance** items. For example, you should exclude windows or animals from the objects you present.
                2. For the coordinates, you should provide them in the form of the top-left and bottom-right points of the bounding box.
                3. If the objects do not fall under the categories of furniture or decorative items, you do not need to present anything.
                4. The coordinates must be presented as relative values to original width and height, ranging from 0 to 1. Do not use absolute values like 210, 850.
                5. The output must be presented in the format specified, listing the object, top-left coordinate, and bottom-right coordinate repeatedly.
                 """}
                ,
                {'type' : 'image_url',
                 'image_url' : url}
            ]           
        )
        
        self.raw_detected = self.lmm.invoke([self.message]).content

        chain = self.prompt | self.llm | self.parser
        raw_axis = chain.invoke({"query" : self.raw_detected})
        raw_axis_upper_left = list(zip(raw_axis.upper_left_axis_x, raw_axis.upper_left_axis_y))
        raw_axis_bottom_right = list(zip(raw_axis.bottom_right_axis_x, raw_axis.bottom_right_axis_y))
        return dict(zip(raw_axis.detected_furnitures, raw_axis_upper_left)),\
        dict(zip(raw_axis.detected_furnitures, raw_axis_bottom_right))

In [ ]:
bbg = BoundingBoxGenerator()
output_bbgs = []
for url in urls:
    output_bbg = bbg.query_img(url = url)
    output_bbgs.append(output_bbg)

In [ ]:
print(output_bbg[0], output_bbg[1])

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import requests
from io import BytesIO

def draw_rectangles(url:str, upper_left_dict:dict, bottom_right_dict:dict):
    
    res = requests.get(url)
    img = Image.open(BytesIO(res.content))
    draw = ImageDraw.Draw(img)

    width = img.width; height = img.height
    font = ImageFont.load_default()    

    for item in upper_left_dict.keys():

        outline_color = 'red'; text_color = 'blue'
        
        upper_left_axis = (int(upper_left_dict[item][0] * width), int(upper_left_dict[item][1] * height))
        bottom_right_axis = (int(bottom_right_dict[item][0] * width), int(bottom_right_dict[item][1] * height))
        draw.rectangle([upper_left_axis, bottom_right_axis], outline = outline_color, width = 2)
        draw.text(upper_left_axis, item, fill = text_color, font = font)

    return img

In [ ]:
for url, output_bbg in zip(urls, output_bbgs):
    img = draw_rectangles(url = url, upper_left_dict = output_bbg[0], bottom_right_dict = output_bbg[1])
    img.save('gpv_'+url.split('/')[-1])